In [2]:
import pandas as pd
import nltk
import re
nltk.download('stopwords')
from bs4 import BeautifulSoup
import string
import contractions
# from pycontractions import Contractions
# cont = Contractions('GoogleNews-vectors-negative300.bin')

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
lemmatizer = WordNetLemmatizer()
cachedStopWords = stopwords.words('english')

def remove_html(text):
    return re.sub(r'http\S+', '', text)

def filter_word(word):
    word = re.sub(r'[-–—]', " ", word)
    return re.sub(r'[^a-zA-Z\s]+', "", word)

def filter_words(text):
    return ' '.join([filter_word(w) for w in text.split()])   

def remove_contractions(text): # contractions has trouble with large data sets
    return ' '.join([contractions.fix(word) for word in text.split()])

# def remove_stopwords(text):
#     words = [w for w in text if w not in stopwords.words('english')]
#     return words

# def word_lemmatizer(text):
#     lem_text = ' '.join([lemmatizer.lemmatize(w) for w in text])
#     return lem_text

# improved parsing time!! went from 13s per 100rows to <1s
def rmStopAndLemmatize(arr):
    return ' '.join([lemmatizer.lemmatize(w) for w in arr if w not in cachedStopWords])


tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adobo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv('train_20210202.csv')


,domain,label,text,Unnamed: 3,Unnamed: 4
0,beforeitsnews.com,fake,"Satanism in the 21st Century: Movie, “The Sour...",NaN,NaN
1,beforeitsnews.com,fake,Is the Gun Rights March To Be Used to Manupula...,NaN,NaN
2,beforeitsnews.com,fake,UPDATE: 5/20/13 – ALERT CRITICAL INFORMATION: ...,NaN,NaN
3,beforeitsnews.com,fake,(Before It's News)\n\nVatic Note:\n\nJefferson...,NaN,NaN
4,beforeitsnews.com,fake,Age: Is Your Food Shortening Your Life?\n\n(Be...,NaN,NaN
...,...,...,...,...,...
18133,thehardtimes.net,satire,A good viral video is like a short story. When...,NaN,NaN
18134,thehardtimes.net,satire,A good viral video is like a short story. When...,NaN,NaN
18135,thehardtimes.net,satire,A great hardcore show is all about energy. For...,NaN,NaN
18136,thehardtimes.net,satire,A great hardcore show is all about energy. For...,NaN,NaN


In [4]:
df = df.drop(columns=['Unnamed: 3', 'Unnamed: 4'])
df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
df

,domain,label,text
0,beforeitsnews.com,fake,"Satanism in the 21st Century: Movie, “The Sour..."
1,beforeitsnews.com,fake,Is the Gun Rights March To Be Used to Manupula...
2,beforeitsnews.com,fake,UPDATE: 5/20/13 – ALERT CRITICAL INFORMATION: ...
3,beforeitsnews.com,fake,(Before It's News)\n\nVatic Note:\n\nJefferson...
4,beforeitsnews.com,fake,Age: Is Your Food Shortening Your Life?\n\n(Be...
...,...,...,...
18133,thehardtimes.net,satire,A good viral video is like a short story. When...
18134,thehardtimes.net,satire,A good viral video is like a short story. When...
18135,thehardtimes.net,satire,A great hardcore show is all about energy. For...
18136,thehardtimes.net,satire,A great hardcore show is all about energy. For...


In [5]:
df['text'] = df['text'].apply(lambda x: remove_html(x))
df['text'] = df['text'].apply(lambda x: remove_contractions(x))
df['text'] = df['text'].apply(lambda x: filter_words(x))
df['text'] = df['text'].apply(lambda x: tokenizer.tokenize(x.lower()))
df['text'] = df['text'].apply(lambda x: rmStopAndLemmatize(x))

df = df[~(df.text.str.len() < 100)]
df.drop_duplicates()
df

,domain,label,text
0,beforeitsnews.com,fake,satanism st century movie source code news rea...
1,beforeitsnews.com,fake,gun right march used manupulate constitutional...
2,beforeitsnews.com,fake,update alert critical information insider yous...
3,beforeitsnews.com,fake,news vatic note jefferson said people fear gov...
4,beforeitsnews.com,fake,age food shortening life news seems every week...
...,...,...,...
18131,thedailymash.co.uk,satire,global nuclear war followed new dark age terro...
18132,thedailymash.co.uk,satire,glut kooky female sitcom character putting wom...
18134,thehardtimes.net,satire,good viral video like short story underdog ris...
18136,thehardtimes.net,satire,great hardcore show energy crowd energy band c...


In [6]:
df = df.drop(columns=['domain'])
df



,label,text
0,fake,satanism st century movie source code news rea...
1,fake,gun right march used manupulate constitutional...
2,fake,update alert critical information insider yous...
3,fake,news vatic note jefferson said people fear gov...
4,fake,age food shortening life news seems every week...
...,...,...
18131,satire,global nuclear war followed new dark age terro...
18132,satire,glut kooky female sitcom character putting wom...
18134,satire,good viral video like short story underdog ris...
18136,satire,great hardcore show energy crowd energy band c...


In [7]:
df.to_csv('train_20210202_cleaned.csv')

In [15]:
df = pd.read_csv('train_20210202_cleaned.csv')
df

,label,text
0,political,abandoning world bank pitfall right left agree...
1,reliable,abc decision cancel popular comedy series last...
2,political,abc journalist foleys family threatened wprose...
3,reliable,abc news report illegal immigrant population u...
4,reliable,abc post poll released sunday found president ...
...,...,...
16382,reliable,zika transmission confirmed two neighborhood m...
16383,reliable,zika virus caught world attention last year ex...
16384,conspiracy,zodiac actually hold quite bit wisdom u said b...
16385,conspiracy,zodiac sign able tell u much realize come love...


In [16]:
df.drop_duplicates(keep=False, inplace=True)

In [17]:
df

,label,text
0,political,abandoning world bank pitfall right left agree...
1,reliable,abc decision cancel popular comedy series last...
2,political,abc journalist foleys family threatened wprose...
3,reliable,abc news report illegal immigrant population u...
4,reliable,abc post poll released sunday found president ...
...,...,...
16382,reliable,zika transmission confirmed two neighborhood m...
16383,reliable,zika virus caught world attention last year ex...
16384,conspiracy,zodiac actually hold quite bit wisdom u said b...
16385,conspiracy,zodiac sign able tell u much realize come love...


In [14]:
df.to_csv('train_20210202_cleaned.csv')